# Option trade

In this assignment, you will make a market on an options spread and estimate its risk. You may pick a reasonable constant
interest rate for option pricing, but all other pricing parameters should use market or market-implied values.


In [7]:
import os
import sys

# From root/notebooks -> go one level up to root
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

ROOT

'/Users/adithsrinivasan/Documents/GitHub/finm37000-2025'

In [24]:
import databento as db
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm

from src.finm37000 import get_databento_api_key, temp_env
from src.finm37000.options import (
    get_options_chain,
    get_top_of_book,
    imply_american_vols,
    imply_european_vol,
)
from src.finm37000.time import tz_chicago

from src.finm37000.skew import (
    calculate_option_vols,
    filter_otm,
    fit_weighted_piecewise_polynomial_skew,
)

from src.finm37000.plotting import (
    add_vol_plot,
    add_vol_range,
    add_underlying,
    layout_vol,
)

In [14]:
with temp_env(DATABENTO_API_KEY=get_databento_api_key(f"{ROOT}/.databento_api_key")):
    client = db.Historical()

1. You are market-making soybean options. It is 12:00 Chicago time on 2025-11-04.
An RFQ (https://www.cmegroup.com/education/request-for-quote.html) comes in
for the OZSF6 C1170 - P1070 risk reversal (https://cmegroupclientsite.atlassian.net/wiki/spaces/EPICSANDBOX/pages/457089763/Spreads+and+Combinations+Available+on+CME+Globex#RR-Risk-Reversal)

Check the market on the legs of this spread and provide your market on this RFQ (size and price for both bid and ask).


In [15]:
# RFQ Parameters

as_of = pd.Timestamp("2025-11-04T12:00:00", tz=tz_chicago)

parent_option = "OZS"   # Soybean options parent product on CME (adjust if your setup uses something else)
underlying_symbol = "ZSF6"  # Soybean futures underlying for OZSF6
option_label = "OZSF6"

call_strike = 1170.0
put_strike = 1070.0

In [16]:
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=as_of,
    client=client,
)

start = as_of
end = as_of + pd.Timedelta(minutes=1)

top_prices = get_top_of_book(
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=start,
    end=end,
    client=client,
)

# Join top-of-book onto the option chain
with_top = options_chain.join(top_prices, on="symbol").dropna(subset=["midprice"])

# --- Extract the two legs: C1170 and P1070 ---

call_leg = with_top[
    (with_top["instrument_class"] == "C")
    & np.isclose(with_top["strike_price"], call_strike)
].iloc[0]

put_leg = with_top[
    (with_top["instrument_class"] == "P")
    & np.isclose(with_top["strike_price"], put_strike)
].iloc[0]

call_leg, put_leg

(ts_event             2025-11-02 12:00:25.290000+00:00
 rtype                                              19
 publisher_id                                        1
 instrument_id                                42579924
 raw_symbol                                OZSF6 C1170
                                    ...               
 ask                                              8.75
 midprice                                        8.625
 bidq                                            193.0
 askq                                            581.0
 weighted_midprice                            8.562339
 Name: 2025-11-04 00:00:00+00:00, Length: 71, dtype: object,
 ts_event             2025-11-02 12:00:18.240000+00:00
 rtype                                              19
 publisher_id                                        1
 instrument_id                                42119935
 raw_symbol                                OZSF6 P1070
                                    ...               
 ask

In [18]:
# Natural RFQ market: OZSF6 C1170 - P1070

rr_bid_natural = call_leg["bid"] - put_leg["ask"]
rr_ask_natural = call_leg["ask"] - put_leg["bid"]

size_bid_natural = min(call_leg["bidq"], put_leg["askq"])
size_ask_natural = min(call_leg["askq"], put_leg["bidq"])

print(f"Natural RR bid: {rr_bid_natural}, size: {size_bid_natural}")
print(f"Natural RR ask: {rr_ask_natural}, size: {size_ask_natural}")


Natural RR bid: 0.375, size: 193.0
Natural RR ask: 0.875, size: 581.0


2. Many traders refer to risk reversals by delta rather than strike. Which delta risk reversal is this? This is not meant to be overly
precise, typically a multiple of 5 would suffice.

You can do this with or without the skew-adjusted delta. There could be some discrepancy depending on how much skew-adjustment
there is, whether you fit to mid-market, etc. All are acceptable answers that should not differ much.

In [20]:
# Mid of the underlying futures
underlying_price = top_prices.loc[underlying_symbol, "midprice"]

# Attach bid/ask/mid etc. to each option
with_top_prices = options_chain.join(top_prices, on="symbol").dropna(subset=["midprice"])

# Add underlying price and a chosen risk‑free rate (e.g. 4%)
with_top_prices["underlying_price"] = underlying_price
with_top_prices["interest_rate"] = 0.04

# Implied vols from American model on bid/mid/ask/weighted_midprice
with_vols = with_top_prices.assign(
    **imply_american_vols(
        with_top_prices,
        futures_price=underlying_price,
        risk_free_rate=0.04,
    )
)

# European Black‑76 implied vol from midprice
with_vols["european_vol"] = with_vols.apply(imply_european_vol, axis=1)

with_vols.head()

vol_col = "european_vol"   # or "iv_midprice" if you prefer BAW vols
r = with_vols["interest_rate"].iloc[0]
F = with_vols["underlying_price"].iloc[0]

# 2. Grab the rows for C1170 and P1070
call_row = with_vols[
    (with_vols["instrument_class"] == "C")
    & np.isclose(with_vols["strike_price"], 1170.0)
].iloc[0]

put_row = with_vols[
    (with_vols["instrument_class"] == "P")
    & np.isclose(with_vols["strike_price"], 1070.0)
].iloc[0]

Kc, Tc, sig_c = call_row["strike_price"], call_row["years_to_expiration"], call_row[vol_col]
Kp, Tp, sig_p = put_row["strike_price"], put_row["years_to_expiration"], put_row[vol_col]

# 3. Compute Black‑76 deltas for each leg
def black_forward_delta(F, K, T, vol, is_call: bool) -> float:
    d1 = (np.log(F / K) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
    cp = 1.0 if is_call else -1.0
    return cp * norm.cdf(cp * d1)  # ∂(option)/∂F

delta_call = black_forward_delta(F, Kc, Tc, sig_c, True)
delta_put  = black_forward_delta(F, Kp, Tp, sig_p, False)

delta_call, delta_put


Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 1240.0: lower_vol=121.67887257706461 target=117.75 upper_vol=706.6688380065184
  F=1117.625 T=0.1426179604261796 r=0.04 mid=117.75
Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 1290.0: lower_vol=171.39444870661094 target=166.375 upper_vol=746.3800298712898
  F=1117.625 T=0.1426179604261796 r=0.04 mid=166.375


(np.float64(0.2302862254818865), np.float64(-0.21481408562581034))

In [21]:
rr_delta = 0.5 * (abs(delta_call) + abs(delta_put))
rr_delta_label = 5 * round(100 * rr_delta / 5)   # round to nearest 5%
rr_delta, rr_delta_label

(np.float64(0.2225501555538484), 20)

Average absolute delta ≈ (0.2303 + 0.2148)/2 ≈ 0.2225 → about 22%. Rounded to the usual 5‑delta multiples, you’d describe this as roughly a 25‑delta risk reversal.

3. Suppose your bid is hit, and you want to understand the risk of you new position. Fit a skew model to the market.

In [25]:
interest_rate = 0.04  # same as before

# Compute implied vols for all options at the RFQ time
with_vols, underlying_price = calculate_option_vols(
    top_df=top_prices,
    underlying_symbol=underlying_symbol,
    option_chain=options_chain,
    interest_rate=interest_rate,
)

# lice to the maturity of OZSF6 C1170 / P1070
rr_expiry = with_vols.loc[
    (with_vols["instrument_class"] == "C")
    & np.isclose(with_vols["strike_price"], 1170.0),
    "expiration",
].iloc[0]

same_expiry = with_vols[with_vols["expiration"] == rr_expiry]
otm_options = filter_otm(same_expiry, underlying_price)

# Fit a (piecewise) polynomial skew to midprice IVs
vol_target = "iv_midprice"
degree = 4  # quadratic or quartic are common choices

skew_func = fit_weighted_piecewise_polynomial_skew(
    otm_options["strike_price"].to_numpy(),
    otm_options[vol_target].to_numpy(),
    atm=underlying_price,
    degree=degree,
)

otm_options["skew_vol"] = skew_func(otm_options["strike_price"])


Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 1240.0: lower_vol=121.67887257706461 target=117.75 upper_vol=706.6688380065184
  F=1117.625 T=0.1426179604261796 r=0.04 mid=117.75
Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 1290.0: lower_vol=171.39444870661094 target=166.375 upper_vol=746.3800298712898
  F=1117.625 T=0.1426179604261796 r=0.04 mid=166.375


In [26]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name="OTM midprice IV",
    y_col="iv_midprice",
)
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name=f"Deg {degree} piecewise skew",
    y_col="skew_vol",
)
add_vol_range(fig, vol_df=otm_options)
add_underlying(fig, underlying_price=underlying_price)
layout_vol(fig, label="OZSF6", detail="Piecewise polynomial skew")
fig.show()


4. State your assumption about how your skew model will move with the underlying price.
Plot the delta and vega of the risk reversal with this skew model over a range of
`[1000, 1200]`.

I assume that implied volatility stays fixed at each strike level (e.g., σ(1170) for the 1170C, σ(1070) for the 1070P) regardless of underlying futures price movements.

In [27]:
call_row = same_expiry[
    (same_expiry["instrument_class"] == "C")
    & np.isclose(same_expiry["strike_price"], call_strike)
].iloc[0]
put_row = same_expiry[
    (same_expiry["instrument_class"] == "P")
    & np.isclose(same_expiry["strike_price"], put_strike)
].iloc[0]

T = float(call_row["years_to_expiration"])
r = interest_rate

In [28]:
call_vol = float(skew_func(np.array([call_strike]))[0])
put_vol = float(skew_func(np.array([put_strike]))[0])

def black_delta_vega(F, K, T, vol, is_call: bool) -> tuple[float, float]:
    """Black‑76 delta and vega w.r.t. forward F."""
    d1 = (np.log(F / K) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
    discount = np.exp(-r * T)
    cp = 1.0 if is_call else -1.0
    delta = cp * norm.cdf(cp * d1)                # ∂price/∂F
    vega = discount * F * np.sqrt(T) * norm.pdf(d1)  # ∂price/∂vol
    return delta, vega

# 3. Compute delta and vega of the risk reversal over F in [1000, 1200]
F_grid = np.linspace(1000.0, 1200.0, 41)
rr_delta = []
rr_vega = []

for F in F_grid:
    d_c, v_c = black_delta_vega(F, call_strike, T, call_vol, True)
    d_p, v_p = black_delta_vega(F, put_strike, T, put_vol, False)
    # Our bid was hit → long RR: +C1170 - P1070
    rr_delta.append(d_c - d_p)
    rr_vega.append(v_c - v_p)


In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=F_grid, y=rr_delta, name="Delta"))

fig.add_trace(
    go.Scatter(
        x=F_grid,
        y=rr_vega,
        name="Vega",
        yaxis="y2",
    )
)

fig.update_layout(
    title="OZSF6 C1170 - P1070 Risk Reversal: Delta & Vega vs Underlying",
    xaxis_title="Underlying futures price (F)",
    yaxis_title="Delta",
    yaxis2=dict(
        title="Vega",
        overlaying="y",
        side="right",
    ),
    template="plotly_white",
)
fig.show()

5. What is the P&L of this position one day later?

In [30]:
call_strike = 1170.0
put_strike = 1070.0
position_size = 100          # spreads (you chose this earlier)
contract_multiplier = 50.0   # $ per 1 cent move for soybeans (5,000 bu * $0.01)

t0 = as_of
t1 = as_of + pd.Timedelta(days=1)   # one calendar day later, same time

call_symbol = options_chain.loc[
    (options_chain["instrument_class"] == "C")
    & np.isclose(options_chain["strike_price"], call_strike),
    "raw_symbol",
].iloc[0]

put_symbol = options_chain.loc[
    (options_chain["instrument_class"] == "P")
    & np.isclose(options_chain["strike_price"], put_strike),
    "raw_symbol",
].iloc[0]

symbols = [underlying_symbol, call_symbol, put_symbol]

top_t0 = get_top_of_book(
    symbols=symbols,
    start=t0,
    end=t0 + pd.Timedelta(minutes=1),
    client=client,
)
top_t1 = get_top_of_book(
    symbols=symbols,
    start=t1,
    end=t1 + pd.Timedelta(minutes=1),
    client=client,
)

call_mid_t0 = top_t0.loc[call_symbol, "midprice"]
put_mid_t0 = top_t0.loc[put_symbol, "midprice"]
call_mid_t1 = top_t1.loc[call_symbol, "midprice"]
put_mid_t1 = top_t1.loc[put_symbol, "midprice"]

rr_mid_t0 = call_mid_t0 - put_mid_t0
rr_mid_t1 = call_mid_t1 - put_mid_t1

print("RR mid at entry:", rr_mid_t0)
print("RR mid one day later:", rr_mid_t1)

RR mid at entry: 0.625
RR mid one day later: 6.375


In [ ]:
trade_px = rr_bid_natural  # the bid price you quoted in Q1
pnl_per_spread = (rr_mid_t1 - trade_px) * contract_multiplier
total_pnl = pnl_per_spread * position_size

total_pnl

6. How well did your skew model approximate this P&L? That is, if you evaluated the price of the risk reversal at the entry time with your fitted skew
but with the underlying and time-to-expiration shifted to their values one day later, how close is this to the actual P&L?

N.B., it is common for traders to use delta and theta to mentally approximate/validate their risk. You may use delta and theta
to get your approximation, but you can also use the skew-modeled price change, which is the common way to do
the calculation in software.
